In [ ]:
import numpy as np


In [ ]:
data4 = np.load('mode4_data.npz')
data15 = np.load('mode15_data.npz')

x_4 = data4['x']
mode_4 = data4['mode']

x_15 = data15['x']
mode_15 = data15['mode']

In [ ]:
#Single deck
def mode_shape_single(mode_4, mode_15, full_matrix=True):
    """
    Constructs mode shape matrices for a single-deck bridge using vertical (mode 4)
    and torsional (mode 15) modes.

    Parameters:
        mode_4 (np.ndarray): Mode shape data for vertical mode (shape: [N, 6]).
        mode_15 (np.ndarray): Mode shape data for torsional mode (shape: [N, 6]).
            - x ranges from -654 to 654 (N)
            - Each row contains [x, y, z, dx, dy, dz] at each node
        full_matrix (bool): If True, populate full 2x2 matrix. If False, keep only diagonal.

    Returns:
        phi_single (np.ndarray): Array of shape (N, 2, 2) containing mode shape matrices per node.
    """
    N = mode_4.shape[0]
    phi_single = np.zeros((N, 2, 2))

    # Extract vertical and rotational components from mode data
    phi_z_1V = mode_4[:, 2]      # Vertical displacement (z) from mode 4
    phi_theta_1V = mode_4[:, 3]  # Rotation (theta) from mode 4
    phi_z_1T = mode_15[:, 2]     # Vertical from torsional mode
    phi_theta_1T = mode_15[:, 3] # Rotation from torsional mode

    for i, (z1V, t1V, z1T, t1T) in enumerate(zip(phi_z_1V, phi_theta_1V, phi_z_1T, phi_theta_1T)):
        phi_single[i, 0, 0] = z1V
        phi_single[i, 1, 1] = t1T

        if full_matrix:
            phi_single[i, 1, 0] = t1V
            phi_single[i, 0, 1] = z1T

    return phi_single


In [ ]:
#Double deck
def mode_shape_double(mode_4, mode_15, full_matrix=True):
    """
    Constructs mode shape matrices for a twin-deck bridge based on mode 4 (vertical) and
    mode 15 (torsional), assuming same mode shapes for both decks.

    Parameters:
        mode_4 (np.ndarray): Mode shape data for vertical mode (shape: [N, 6]).
        mode_15 (np.ndarray): Mode shape data for torsional mode (shape: [N, 6]).
            - x ranges from -654 to 654 (N)
            - Each row contains [x, y, z, dx, dy, dz] at each node
        full_matrix (bool): If True, populate full 4x4 matrices. If False, only diagonals.

    Returns:
        phi_double (np.ndarray): Array of shape (N, 4, 4) with mode shape matrices per node.
                                 DOFs are [z1, θ1, z2, θ2].
    """
    N = mode_4.shape[0]
    phi_double = np.zeros((N, 4, 4))

    # Extract relevant DOFs from each mode
    phi_z_1V = mode_4[:, 2]       # vertical deck 1
    phi_theta_1V = mode_4[:, 3]   # torsion deck 1
    phi_z_1T = mode_15[:, 2]      # vertical (torsion mode) deck 1
    phi_theta_1T = mode_15[:, 3]  # torsion (torsion mode) deck 1

    for i, (z1V, t1V, z1T, t1T) in enumerate(zip(phi_z_1V, phi_theta_1V, phi_z_1T, phi_theta_1T)):
        # Deck 1
        phi_double[i, 0, 0] = z1V
        phi_double[i, 1, 1] = t1T

        # Deck 2 (mirrored)
        phi_double[i, 2, 2] = z1V
        phi_double[i, 3, 3] = t1T

        if full_matrix:
            phi_double[i, 1, 0] = t1V
            phi_double[i, 0, 1] = z1T
            phi_double[i, 3, 2] = t1V
            phi_double[i, 2, 3] = z1T

    return phi_double